In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.5/756.5 kB 10.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-man

In [4]:
!pip install tracker

  Preparing metadata (setup.py) ... done
  Created wheel for tracker: filename=tracker-0.1.1-py3-none-any.whl size=3113 sha256=3c6a507858cd1b0b656796419f5df984cd2e945433b453fa40d4ead3cad4eeac
  Stored in directory: /root/.cache/pip/wheels/54/ea/0a/d9a06e1e36873dfda7485e4fc576791f4afb5442116ae359d9
Successfully built tracker


In [10]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
import pandas as pd
from tracker import *
import math
from google.colab import files


model = YOLO('/content/drive/My Drive/Colab Notebooks_edge/best_segment_old.pt')

model_person = YOLO('/content/drive/My Drive/Colab Notebooks_edge/yolov8s.pt')

class_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

class Tracker:
    def __init__(self):
        # Store the center positions of the objects
        self.center_points = {}
        # Keep the count of the IDs
        # each time a new object id detected, the count will increase by one
        self.id_count = 0

    def update(self, objects_rect):
        # Objects boxes and ids
        objects_bbs_ids = []

        # Get center point of new object
        for rect in objects_rect:
            x, y, w, h = rect
            cx = (x + x + w) // 2
            cy = (y + y + h) // 2

            # Find out if that object was detected already
            same_object_detected = False
            for id, pt in self.center_points.items():
                dist = math.hypot(cx - pt[0], cy - pt[1])

                if dist < 35:
                    self.center_points[id] = (cx, cy)
                    objects_bbs_ids.append([x, y, w, h, id])
                    same_object_detected = True
                    break

            # New object is detected we assign the ID to that object
            if same_object_detected is False:
                self.center_points[self.id_count] = (cx, cy)
                objects_bbs_ids.append([x, y, w, h, self.id_count])
                self.id_count += 1

        # Clean the dictionary by center points to remove IDS not used anymore
        new_center_points = {}
        for obj_bb_id in objects_bbs_ids:
            _, _, _, _, object_id = obj_bb_id
            center = self.center_points[object_id]
            new_center_points[object_id] = center

        # Update dictionary with IDs not used removed
        self.center_points = new_center_points.copy()
        return objects_bbs_ids

def detect_person(frame, image, inc_start, dec_start):
    results_person = model_person.predict(frame)
    a = results_person[0].boxes.data
    a = a.detach().cpu().numpy()  # added this line
    px = pd.DataFrame(a).astype("float")

    detected_persons = []
    for index, row in px.iterrows():
        x1_person, y1_person, x2_person, y2_person, d_person = int(row[0]), int(row[1]), int(row[2]), int(row[3]), int(row[5])
        c_person = class_list[d_person]
        if 'person' in c_person:
            detected_persons.append([x1_person, y1_person, x2_person, y2_person])

    bbox_id = tracker.update(detected_persons)
    for bbox in bbox_id:
        x1_person, y1_person, x2_person, y2_person, id_person = bbox
        cv2.rectangle(image, (x1_person, y1_person), (x2_person, y2_person), (0, 255, 0), 2)  # Draw rectangle around detected persons
        print("Rectangle Coordinates:", (x1_person, y1_person), (x2_person, y2_person))  # Print coordinates of the rectangle

        if inc_start is not None and dec_start is not None:
          if inc_start>dec_start:
              cv2.circle(image, (x1_person, y2_person), 1, (0,0,128), thickness=4)

          elif inc_start<dec_start:
              cv2.circle(image, (x2_person, y2_person), 1, (0,0,128), thickness=4)

    return detected_persons , image

def find_dec_and_inc(dec_start,dec_end,inc_start,inc_end,mask_points):


        dec_start_temp = -1
        dec_end_temp = -1
        inc_start_temp = -1
        inc_end_temp = -1

        max_dec_length = 0
        max_inc_length = 0

        # Iterate through the mask points to find the decreasing interval
        i = 0
        s = 0
        while s < len(mask_points) - 1:
            if mask_points[s][0] > mask_points[s + 1][0]:
                dec_start_temp = s
                i=s
                while i < len(mask_points) - 1 and mask_points[i][0] >= mask_points[i + 1][0]:
                    i += 1
                dec_end_temp = i
                dec_length = dec_end - dec_start + 1
                if dec_length > max_dec_length:
                    max_dec_length = dec_length
                    dec_end=dec_end_temp
                    dec_start=dec_start_temp
            s += 1

        # Iterate through the mask points to find the increasing interval
        i = 0
        s=0
        while s < len(mask_points) - 1:
            if mask_points[s][0] < mask_points[s + 1][0]:
                inc_start_temp = s
                i=s
                while i < len(mask_points) - 1 and mask_points[i][0] <= mask_points[i + 1][0]:
                    i += 1
                inc_end_temp = i
                inc_length = inc_end - inc_start + 1
                if inc_length > max_inc_length:
                    max_inc_length = inc_length
                    inc_end=inc_end_temp
                    inc_start=inc_start_temp
            s += 1

        return dec_start ,dec_end ,inc_start, inc_end

def check_for_track(p, q, r, s, x1,y1,x2,y2, dec_start, dec_end, inc_start, inc_end, mask_points,image):
    flag_on_track=0
    X1=0
    X2=0
    Y1=0
    Y2=0
    flag=0
    point2 = None
    point1 = None

    if inc_start<dec_start:

        z=inc_start
        for z in range(inc_start,inc_end-1):
            if mask_points[z][0]>=(p-30) and mask_points[z][0]<=(p+30):
                flag=1
                point1 = tuple(mask_points[z])
                X1=mask_points[z][0]
                Y1=mask_points[z][1]
                break

        if flag==0:
            point1 = tuple(mask_points[inc_end-20])
            X1=mask_points[inc_end-20][0]
            Y1=mask_points[inc_end-20][1]

        print("starting point is",point1)

        for w in range(z+1,inc_end-1):
            if mask_points[w][0]<=(p+30) and mask_points[w][0]!=X1:
                point2 = tuple(mask_points[w - 1])
                X2=mask_points[w-1][0]
                Y2=mask_points[w-1][1]

        if X2<(p-30) or X2>(p+30):
            point2 = tuple(mask_points[inc_end-10])
            X2=mask_points[inc_end-10][0]
            Y2=mask_points[inc_end-10][1]

        print("Ending point is ",point2)


        cv2.line(image, point1, point2, (0, 0, 128), thickness=4)

        #X2,X,Y1,Y2 are coordinates of mask_points
        if p>x1 and p<x2 and q>y1 and q<y2:
            if X2-X1!=0:
                if (Y2 - Y1) / (X2 - X1)!=0:
                    m = (Y2 - Y1) / (X2 - X1)
                    print(m)
                    Cx=p
                    Cy=q
                    print("checking")
                    print(Cy - m * (Cx - X1) - Y1)
                    if Cy - m * (Cx - X1) - Y1 == 0 or Cy - m * (Cx - X1) - Y1 < 0:
                        print("mask point 1st point ",X1,Y1)
                        print("mask points 2nd point ",X2,Y2)
                        flag_on_track=1

        X3=0
        X4=0
        Y3=0
        Y4=0
        flag=0
        if(flag_on_track==1):

            print("Entered into track now check for 2nd line")
            w=dec_start
            for w in range(dec_start,dec_end):
                if mask_points[w][0]<=(p+30):
                    flag=1
                    point2 = tuple(mask_points[w])
                    X4=mask_points[w][0]
                    Y4=mask_points[w][1]
                    break

            if flag==0:
                point2 = tuple(mask_points[dec_end-20])
                X4=mask_points[dec_end-20][0]
                Y4=mask_points[dec_end-20][1]

            print("Ending point of 2nd line",point2)

            flag=0
            for z in range(w+1,dec_end):
                if mask_points[z][0]>=(p-30) and mask_points[z][0]<=(p+30):
                    flag=1
                    point1 = tuple(mask_points[z])
                    X3=mask_points[z][0]
                    Y3=mask_points[z][1]

            if flag==0:
                point1=tuple(mask_points[dec_end-10])
                X3=mask_points[dec_end-10][0]
                Y3=mask_points[dec_end-10][1]

            print("starting point for 2nd line ",point1)



            cv2.line(image, point1, point2, (255, 255, 0), thickness=4)

            if p>x1 and p<x2 and q>y1 and q<y2:
                if X4-X3!=0:
                    if (Y4-Y3)/(X4-X3)!=0:
                        m=(Y4-Y3)/(X4-X3)
                        Cx=p
                        Cy=q
                        if Cy - m * (Cx - X3) - Y3 == 0 or Cy - m * (Cx - X3) - Y3 > 0:
                            print("you are on the track")
                            text_x = (p + r) // 2
                            text_y = (q + s) // 2
                            cv2.putText(image, "Detected", (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)


    #########################################################################################################
    #########################################################################################################
    flag_on_track=0
    X1=0
    X2=0
    Y1=0
    Y2=0
    flag=0
    point2 = None
    point1 = None
    if inc_start>dec_start:
        z=inc_start
        for z in range(inc_start,inc_end):
            if mask_points[z][0]>=(p-30) and mask_points[z][0]<=(p+30):
                flag=1
                point1 = tuple(mask_points[z])
                X1=mask_points[z][0]
                Y1=mask_points[z][1]
                break

        if flag==0:
            point1 = tuple(mask_points[inc_start+10])
            X1=mask_points[inc_start+10][0]
            Y1=mask_points[inc_start+10][1]

        print("starting point is",point1)

        for w in range(z+1,inc_end):
            if mask_points[w][0]<=(p+30):
                point2 = tuple(mask_points[w - 1])
                X2=mask_points[w-1][0]
                Y2=mask_points[w-1][1]

        if X2<(p-30) or X2>(p+30):
            point2 = tuple(mask_points[inc_start+20])
            X2=mask_points[inc_start+20][0]
            Y2=mask_points[inc_start+20][1]



        print("Ending point is ",point2)

        cv2.line(image, point1, point2, (0, 0, 128), thickness=4)

            #X2,X,Y1,Y2 are coordinates of mask_points
        if p>x1 and p<x2 and q>y1 and q<y2:
            if X2-X1!=0:
                if (Y2-Y1)/(X2-X1)!=0:
                    m = (Y2 - Y1) / (X2 - X1)
                    print(m)
                    Cx=p
                    Cy=q
                    print("checking")
                    print(Cy - m * (Cx - X1) - Y1)
                    if Cy - m * (Cx - X1) - Y1 == 0 or Cy - m * (Cx - X1) - Y1 < 0:
                        print("mask point 1st point ",X1,Y1)
                        print("mask points 2nd point ",X2,Y2)
                        flag_on_track=1
        X3=0
        X4=0
        Y3=0
        Y4=0

        if(flag_on_track==1):
            print("Entered into track now chech for 2nd line")
            w=dec_start
            for w in range(dec_start,dec_end):
                if mask_points[w][0]<=(p+30):
                    point2 = tuple(mask_points[w])
                    X4=mask_points[w][0]
                    Y4=mask_points[w][1]
                    break
            print("Ending point of 2nd line",point2)
            flag=0
            for z in range(w+1,dec_end):
                if mask_points[z][0]>=(p-30) and mask_points[z][0]<=(p+30):
                    flag=1
                    point1 = tuple(mask_points[z])
                    X3=mask_points[z][0]
                    Y3=mask_points[z][1]

            if flag==0:
                point1=tuple(mask_points[w+10])
                X3=mask_points[w+10][0]
                Y3=mask_points[w+10][1]



            print("starting point for 2nd line ",point1)



            cv2.line(image, point1, point2, (255, 255, 0), thickness=4)

            if p>x1 and p<x2 and q>y1 and q<y2:
                if X4-X3!=0:
                    if (Y4-Y3)/(X4-X3)!=0:
                        m=(Y4-Y3)/(X4-X3)
                        Cx=p
                        Cy=q
                        if Cy - m * (Cx - X3) - Y3 == 0 or Cy - m * (Cx - X3) - Y3 > 0:
                            cv2.circle(image, (p, q), 1, (0,0,128), thickness=2)
                            print("you are on the track")
                            print("you are on the track")
                            text_x = (p + r) // 2
                            text_y = (q + s) // 2
                            cv2.putText(image, "Detected", (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    return image

def detect_metro(frame):
    results = model(frame, save=False, save_conf=False, conf=0.5)
    print("Detection results:", results)
    if results is None:
        # If detection fails, return None for all outputs
        return None, None, None, None, None, None, None, None, None, None
    # Extract bounding boxes, classes, object names, confidence scores, and masks
    boxes = results[0].boxes.xyxy.tolist()
    classes = results[0].boxes.cls.tolist()
    confidences = results[0].boxes.conf.tolist()
    masks = results[0].masks

    h, w = results[0].orig_shape

    if masks is None:
        # If masks are None, return None for mask_points and other outputs
        return None, None, None, None, None, None, None, None, None, None

    # Iterate through each detected object's box, class, confidence, and mask
    for i, tup in enumerate(zip(boxes, classes, confidences, masks)):
        box, cls, conf, mask = tup

        # Create a copy of the original frame
        image = frame.copy()

        mask_points = mask.xy[0].astype(int)
        mask_points = np.append(mask_points, [mask_points[0]], axis=0)  # Add the first point to close the polygon

        # Draw bounding box
        x_values = [point[0] for point in mask_points]
        x1 = min(x_values)
        x2 = max(x_values)
        y_values = [point[1] for point in mask_points]
        y1 = min(y_values)
        y2 = max(y_values)
        cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)

        # Draw mask polygon
        for i in range(len(mask_points) - 1):
            point1 = tuple(mask_points[i])
            point2 = tuple(mask_points[i + 1])
            cv2.line(image, point1, point2, (0, 128, 0), thickness=4)

        # Find intervals and perform tracking (as per your original code)
        dec_start = -1
        dec_end = -1
        inc_start = -1
        inc_end = -1

        dec_start, dec_end, inc_start, inc_end = find_dec_and_inc(dec_start, dec_end, inc_start, inc_end, mask_points)

        print("Longest decreasing interval (starting, ending) =", (dec_start, dec_end))
        print("Longest increasing interval (starting, ending) =", (inc_start, inc_end))
        print("Total mask_points:", len(mask_points))
        print("Coordinates for longest decreasing interval =", mask_points[dec_start], mask_points[dec_end])
        print("Coordinates for longest increasing interval =", mask_points[inc_start], mask_points[inc_end])

    return image, mask_points, dec_start, dec_end, inc_start, inc_end,x1,y1,x2,y2



tracker = Tracker()
video_path = '/content/drive/My Drive/Colab Notebooks_edge/PF_EdgeCrossing_5.avi'
cap = cv2.VideoCapture(video_path)

# Define codec and VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (1800, 850))

# Loop through the frames
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (1800, 850))
    processed_frame, mask_points, dec_start, dec_end, inc_start, inc_end, x1, y1, x2, y2 = detect_metro(frame)

    if mask_points is None or dec_start is None or dec_end is None or inc_start is None or inc_end is None or x1 is None or y1 is None or x2 is None or y2 is None:
        # Handle the case where object detection failed
        # For example, you can skip processing this frame or display a message
        out.write(processed_frame)
        print("Object detection failed for this frame.")
        continue

    detected_persons, again_processed_frame = detect_person(frame, processed_frame, inc_start, dec_start)

    # Perform tracking and other operations based on conditions
    if inc_start is not None and dec_start is not None:
      if inc_start > dec_start:
          for i in range(len(detected_persons)):
              x1_person, y1_person, x2_person, y2_person = detected_persons[i]
              again_processed_frame1 = check_for_track(x1_person, y2_person, x2_person, y1_person, x1, y1, x2, y2, dec_start, dec_end, inc_start, inc_end, mask_points, again_processed_frame)
      elif inc_start < dec_start:
          for i in range(len(detected_persons)):
              x1_person, y1_person, x2_person, y2_person = detected_persons[i]
              again_processed_frame1 = check_for_track(x2_person, y2_person, x1_person, y1_person, x1, y1, x2, y2, dec_start, dec_end, inc_start, inc_end, mask_points, again_processed_frame)

    detected_persons.clear()

    # Write the processed frame to the output video file
    out.write(again_processed_frame1)

# Release the VideoCapture and VideoWriter objects
cap.release()
out.release()

cv2.destroyAllWindows()

# Specify the file name you want to download
file_name = "output.avi"

# Download the file
files.download(file_name)

Streaming output truncated to the last 5000 lines.
Coordinates for longest decreasing interval = [1442  169] [1434  194]
Coordinates for longest increasing interval = [1209  247] [1797  301]

0: 320x640 4 persons, 1 train, 10.6ms
Speed: 2.3ms preprocess, 10.6ms inference, 1.4ms postprocess per image at shape (1, 3, 320, 640)
Rectangle Coordinates: (904, 47) (930, 118)
Rectangle Coordinates: (927, 44) (952, 121)
Rectangle Coordinates: (1540, 64) (1566, 125)
Rectangle Coordinates: (966, 34) (981, 81)
starting point is (1237, 284)
Ending point is  (1285, 340)
starting point is (1237, 284)
Ending point is  (1285, 340)
starting point is (1515, 633)
Ending point is  (1563, 700)
starting point is (1237, 284)
Ending point is  (1285, 340)

0: 320x640 1 Metro-track, 14.3ms
Speed: 2.5ms preprocess, 14.3ms inference, 2.7ms postprocess per image at shape (1, 3, 320, 640)
Detection results: [ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
ke

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>